# Práctica: Los Juegos del Hambre

ESTUDIANTE: `ESCRIBE TU NOMBRE AQUI`

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

En esta práctica vamos a enfrentarnos a un problema desafiante de clasificación de imágenes, construyendo una red neuronal profunda que sea capaz de clasificar entre diferentes tipos de comida. ¡Que comiencen los Juegos del Hambre!

## Guidelines

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb), y de haber [deactivado otras sesiones que tuvieras activas](https://stackoverflow.com/a/53441194/2436578).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Obtención de datos

Vamos a usar un dataset de imágenes de comida disponible en [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). Para descargarlo, necesitarás crear una cuenta de usuario en Kaggle, y obtener tus credenciales de la API. Puedes hacerlo siguiendo las instrucciones de [esta sección](https://github.com/Kaggle/kaggle-api#api-credentials). ¡Ojo! Tus credenciales de la API no son lo mismo que la contraseña que utilizas para acceder a tu cuenta en Kaggle.

Una vez tengas el fichero JSON con tus credenciales, puedes declararlas en este notebook asignando las variables de entorno adecuadas, de la siguiente manera

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Cuando lo hayas hecho, podrás descargar el dataset a la máquina donde esté corriendo este notebook usando el siguiente comando

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
donde debes indicar el nombre de un directorio válido como "YOUR_LOCAL_FOLDER". Si prefieres descargar los datos en la misma carpeta que este notebook, puedes quitar la parte `-p YOUR_LOCAL_FOLDER` del comando.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Crea tu propia cuenta de Kaggle (si no tienes ya una), obtén tus credenciales, y usa la celda inferior para declarar tu nombre de usuario de Kaggle y tu clave como variables de entorno. A continuación, usa la misma celda para descargar el dataset de imágenes.
    
¡Ojo! Debes mantener estas credenciales en secreto, ya que son personales a tu usuario de Kaggle. Recuerda borrarlas de la celda antes de entregar este notebook.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Revisa ahora la carpeta en la que has descargado los datos. Verás que contiene 3 subdirectorios:

* **training**, contiene las imágenes a utilizar para entrenar el modelo.
* **validation**, contiene imágenes adicionales que podrías usar como datos de entrenamiento adicionales, o para algún tipo de estrategia de validación como Early Stopping.
* **evaluation**, contiene las imágenes que debes utilizar para testear el modelo. Las imágenes de esta carpeta **solo** pueden utilizarse para medir el rendimiento del modelo tras su entrenamiento, y para nada más.

Además de esto, dentro de cada una de estas carpetas encontrarás una subcarpeta para cada una de las 11 clases de comida:

* Bread (panes)
* Dairy product (lácteos)
* Dessert (postres)
* Egg (huevos)
* Fried food (fritos)
* Meat (carnes)
* Noodles-Pasta (pasta)
* Rice (arroz)
* Seafood (pescado y marisco)
* Soup (sopas)
* Vegetable-Fruit (vegetales y frutas)

Esta es una forma estándar de organizar los datasets de imágenes: una carpeta para cada clase. Para facilitar los pasos de procesamiento que vendrán a continuación, vamos a definir algunas variables que nos indiquen dónde están almacenados los diferentes conjuntos de datos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea variables <b>TRAINDIR</b>, <b>VALDIR</b> y <b>TESTDIR</b>, cada una conteniendo una cadena de texto con la ruta al directorio donde están los datos de entrenamiento, validación y evaluación, respectivamente.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

### Reducción de clases

Con el fin de hacer este problema más accesible de cara a la práctica, vamos a centrarnos solo en seis de las clases de comida disponibles: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` y `Meat`. Para ello, se provee el código siguiente, que elimina de los datos descargados las carpetas correspondientes a imágenes de las otras clases.

In [ ]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

## Procesando imágenes desde ficheros

Este dataset de imágenes es grande, con imágenes de mayor resolución que las que hemos utilizado en el tutorial del MNIST, y cada una de ellas teniendo diferentes tamaños y relación de aspecto. Además, mientras que para el MNIST teníamos una función de keras que preparaba los datos para nosotros, en esta ocasión tendremos que realizar el trabajo de carga y procesamiento de las imágenes.

Una forma conveniente de hacer todo este trabajo es a través de la función Keras `image_dataset_from_directory`. Esta función crea un objeto `Dataset` de TensorFlow con todas las imágenes de un directorio, cargándolas en memoria de forma dinámica solo cuando la red neuronal necesita utilizarlas. Esta función también nos permite especificar algunas opciones de preprocesamiento muy útiles.

Por ejemplo, podemos crear un `Dataset` con los datos en la carpeta de training:

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Observa los parámetros que se han utilizado para configurar el dataset:

* El **directorio** desde el que cargar las imágenes.
* Un **tamaño de imagen (image_size)** que se utilizará para redimensionar todas las imágenes cargadas a ese tamaño común, en este caso 32x32 píxeles.
* El **tamaños de los lotes (batch_size)** de imágenes a ser generados. Nótese que definimos aquí este parametro en lugar de en el paso `fit` de la red, como hemos hecho en otros ejercicios, porque el objeto `Dataset` resultante hará uso de esta información para mantener en memoria solo algunos batches de imágenes, ahorrando así memoria.
* El **modo de etiquetado (label_mode)**, esto es, la codificación de las etiquetas a utilizar. `categorical` significa que utilizaremos la ya conocida codificación one-hot.

Un objeto `Dataset` funciona de manera muy similar a un generador de Python, lo que significa que podemos iterar sobre él para obtener batches de imágenes ya preprocesadas. Por ejemplo, el siguiente código inicia un bucle para extraer todos los batches del `Dataset`, nos muestra el contenido el primero, y detiene la iteración.

In [ ]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Podemos ver que, efectivamente, el generador produce un tensor de datos de entrada de las dimensiones apropiadas para poder introducirlo en la red neuronal, y que las salidas también se han codificado correctamente como one-hot.
No obstante, todavía hay un problema con los datos: los valores de los píxeles están en el rango [0, 255], lo cual puede producir problemas de entrenamiento. Resolveremos este punto después, en la definición de la red neuronal, mediante una capa especial. Por ahora vamos a cotinuar, definiendo una función que construya los `Dataset` para los datos de entrenamiento, validación y test.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una función <b>create_datasets</b> que reciba los siguiente parámetros:
    <ul>
      <li><b>traindir</b>: el directorio donde están localizadas las imágenes de entrenamiento.</li>
      <li><b>valdir</b>: el directorio donde están localizadas las imágenes de validación.</li>
      <li><b>testdir</b>: el directorio donde están localizadas las imágenes de test.</li>
      <li><b>image_size</b>: el tamaño que se utilizará para redimensionar todas las imágenes a una resolución común.</li>
      <li><b>batch_size</b>: el tamaño de los batches de imágenes que serán generados.</li>
    </ul>
    La función debe crear objetos `Dataset` para los directorios de entrenamiento, validación y test, y devolver los tres datasets creados como

    return train_dataset, val_dataset, test_dataset
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Probemos que la función que has implementado funciona correctamente

In [ ]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

¡Ahora que tenemos nuestros `Dataset` podemos entrenar una red profunda con ellos! Como ejemplo, vamos a construir una red convolucional extremadamente simple. Nótese cómo hemos añadido una capa especial de preprocesado llamada `Rescaling`, que será la encargada de normalizar los valores de los píxeles al rango [0, 1] cada vez que la red reciba una imagen.

¡Ojo! Esta red tan simple no producirá errores al ejecutar, pero tiene algunos fallos de diseño que deberás corregir cuando crees tu propia red, más adelante en esta práctica.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

El método `fit` de un modelo Keras puede recibir un objeto `Dataset` como datos de entrenamiento, en lugar de un par de tensores (entradas, salidas). Como al construir los `Dataset` ya especificamos el tamaño de batch, no es necesario indicarlo ahora.

In [ ]:
model.fit(train_dataset, epochs=1)

Análogamente, podemos evaluar el rendimiento de nuestro modelo sobre el `Dataset` de test de la siguiente manera

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

Este nivel de acierto puede parecer pobre, pero ten en cuenta que hemos usado un modelo muy simple y que el problema es de 6 clases. ¿Serás capaz de hacerlo mejor?

## Construyendo tu propia red

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Diseña una red neuronal profunda que obtenga el mejor acierto posible sobre los datos de test. Puedes usar los datos de entrenamiento y validación como te parezca mejor, pero <b>sólo</b> puedes usar los datos de test para evaluar el acierto de tu modelo. Debes obtener una red capaz de alcanzar al menos un 40% de acierto sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunas recomendaciones y estrategias que pueden ayudar a mejorar tu diseño de red:

    
- Usa todos los trucos que has aprendido en los ejercicios anteriores: capas convolucionales + pooling, activaciones ReLU, dropout... asegúrate también de utilizar un buen optimizador, con una función de error (loss) adecuada, así como una función de activación en la capa de salida que sea adecuada para esta clase de problema (clasificación multiclase).
- Empieza por redes pequeñas, con un número pequeño de parámetros, de forma que puedas comprobar rápidamente qué tal funcionan. Después, puedes hacer tu red más grande en tres direcciones: mayor tamaño de imágenes de entrada, más capas, y más kernels por capa convolucional o unidades por capa densa. Si aumentas el tamaño de las imágenes de entrada, asegúrate de añadir también más capas Convolution+Pooling, para que así a la capa Flatten solo lleguen imágenes muy pequeñas (10x10 píxeles o menos).
- Configurar los `Dataset` para que carguen imágenes de mayor tamaño puede mejorar significativamente el rendimiento de tu red. Pero ten cuidado, ¡también puedes encontrarte errores de falta de memoria (CUDA memory error) si cargas imágenes a un tamaño demasiado grande! Para esta práctica, un tamaño mayor a 256 puede ser demasiado grande...
- Si observas grandes diferencias de loss entre los datos de entrenamiento y validación o test, prueba a incrementar el nivel de Dropout en las capas Dense.
- ¡Usa los datos de validación! Por ejemplo, usa una <a href="https://keras.io/api/callbacks/early_stopping/">**estrategia de EarlyStopping**</a> para monitorizar el loss de los datos de validación, y así detener el entrenamiento cuando tras un número de épocas esa loss no haya decrecido. Configurar la EarlyStopping para restaurar los mejores parámetros encontrados durante la optimización también puede resultarte útil.
- Si tu red obtiene resultados muy buenos, del orden del 90% o más de acierto en test... sospecha. Es posible que estés mezclando los datos de entrenamiento y test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Como ejercicio avanzado, añade las siguiente estrategias a tu red:

- Usa **técnicas de "image augmentation"** para aumentar artificialmente tu dataset de entrenamiento. Para ello, explora las <a href="https://keras.io/api/layers/preprocessing_layers/image_augmentation/">capas de augmentation disponibles en Keras</a>.
- Usa capas de <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> para facilitar el entrenamiento de la red.
    
Usando estos trucos y los mencionados en el punto anterior, es posible obtener más de un 60% de acierto en el conjunto de test.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Transfer learning

Aunque diseñar nuestra propia red puede producir algunos resultados aceptables, suele ser mejor aprovechar el conocimiento ya existente en una red pre-entrenada. Esto no solo nos lleva a resultados mejores, sino que además nos ahorra mucho tiempo de diseño de la red. Para ello, el módulo [Keras Applications](https://keras.io/api/applications/) contiene varios diseños de redes listos para su uso. Por ejemplo, para hacer uso de la famosa red VGG16 hacemos lo siguiente

In [ ]:
from tensorflow.keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

Por defecto, todas las redes de Keras Applications están precargadas con los pesos que se obtuvieron al entrenar la red sobre el dataset de la competición de [ImageNet](http://www.image-net.org/). Para adaptar la red a nuestro problema, hemos necesitado especificar la resolución de nuestras imágenes (`input_shape`), así como eliminar las capas de salida (`top`) de la red original, dado que nosotros tendremos un número diferente de clases.

Ahora, ¿cómo hacemos para transferir el aprendizaje de esta red? Vamos a ver cómo implementar la estrategia de "bottleneck features". En primer lugar, marcamos el modelo VGG16 como no entrenable, para que sus parámetros se mantengan congelados.

In [ ]:
vgg16_model.trainable = False

Hecho esto, vamos a construir una red neuronal que incluya la VGG16 como una de sus "capas". Es necesario tener en cuenta que la red VGG16 se entrenó realizando una normalización muy específica de las imágenes de entrenamiento, y nosotros debemos seguir ese mismo proceso para que la red se comporte correctamente. Convenientemente, Keras también nos da una funcionalidad para replicar la normalización que la VGG16 necesita.

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

Podemos probar esta normalización con alguna de las imágenes de nuestro dataset

In [ ]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

La normalización realizada por la VGG16 consiste en invertir el orden de los canales de color (RGB -> BGR), y restar los valores medios sobre todo el dataset ImageNet para cada canal de color por separado. Afortunadamente, la función `preprocess_input` que hemos importado hace todo este trabajo por nosotros. Además, podemos incrustar esta función como la primera capa de nuestra red, cumpliendo el papel de la capa `Rescaling` que utilizamos en el apartado anterior. Esto es posible gracias a la capa `Lambda`, que permite construir una capa Keras en base a cualquier función de Tensorflow. De modo que, vamos a comenzar nuestro diseño de red con esta capa.

In [ ]:
from tensorflow.keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Tras esto, podemos añadir toda la red VGG16 como si fuera una nueva capa, y nuestras propias capas después de ella. A continuación tenemos un ejemplo de esta clase de diseño, aunque es importante destacar que es un diseño muy sencillo que contiene algunos fallos; una red real para hacer transfer learning debería tener un diseño mejor.

In [ ]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Observa cómo en el resumen del modelo podemos ver que la red completa tiene millones de parámetros, pero dado que hemos congelado toda la parte de la red perteneciente a la VGG16, solo unos pocos miles de parámetros son entrenables (trainable): aquellos correspondientes a la capa Dense que hemos colocado al final.

Ya podemos compilar y entrenar el modelo a la manera habitual.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando la estrategia "bottleneck" presentada, implementa una red que haga transfer learning desde la red VGG16, con un diseño correcto. Si lo haces adecuadamente, esta red debe obtener mejores resultados que con la red que diseñaste en el apartado anterior, y con al menos un 80% de acierto sobre el conjunto de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunos consejos para mejorar tu diseño de red:
    
- Incluye una o más capas Dense, con sus funciones de activación apropiadas, antes de la capa de salida.
- Intenta usar una capa de tipo [GlobalAveragePooling](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) en lugar de la capa Flatten. Esta capa calcula una media de todos los valores de píxeles para cada canal, y en algunas ocasiones produce mejores resultados que la capa Flatten.
- ¡Y no olvides todos los consejos del apartado anterior! También aplican aquí.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>

Para mejorar aún más los resultados de tu red, utiliza las siguientes ideas:

- Usa las estrategias PRO del ejercicio anterio.
- Prueba otras redes pre-entrenadas de <a href="https://keras.io/api/applications/">Keras Applications</a>, como ResNet, Xception o EfficientNet.
- Usa una estrategia de transfer learning más avanzada, como fine-tuning o una combinación de bottleneck features y fine-tuning. Revisa las diapositivas de clase para saber cómo.
   
Si empleas todos estos trucos, es posible alcanzar más de un 90% de acierto en el conjunto de test.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Informe final y resumen de resultados

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Escribe en la siguiente celda un pequeño informe con:
    <ul>
        <li>Una tabla de resultados, indicando qué diseños de red has probado y qué resultados en test has obtenido. Puede usar un estilo de tabla como el que se muestra abajo.</li>
        <li>De las estrategias y diseños que has ido probando, ¿qué ha funcionado y qué no?</li>
        <li>¿Qué has aprendido con esta práctica?
    </ul>
</font>

***

Ejemplo para la tabla de resultados

|Procesado de imágenes|Diseño de red neuronal|Estrategia de entrenamiento|Acierto en test|
|---------------------|----------------------|---------------------------|---------------|
|Tamaño 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Entrenamiento desde 0|xx%|
|Tamaño 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|